In [22]:
import os
import numpy as np
from numpy import median
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import VotingClassifier

In [23]:
#load the model and preprocess
import pickle
model = pickle.load(open("model.pickle","rb"))
transformers = pickle.load(open("transformer.pickle","rb"))

In [24]:
#data = [655,'Germany','Female',46,7,137145.12,1,'Yes','No',115146.4]
#data=[411,'France','Male',29,0,59697.17,2,1,1,53483.21]
#data=[740,'Germany','Male',33,2,126524.11,1,1,0,136869.31]
data=[725,'France','Female',33,4,0,1,1,1,67879.8]
data  = pd.DataFrame(data).T
data.columns = ["CreditScore","Geography","Gender","Age","Tenure","Balance","NumOfProducts","HasCrCard","IsActiveMember","EstimatedSalary"]
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,725,France,Female,33,4,0,1,1,1,67879.8


In [25]:
def prep_pred(data):
    if(data['HasCrCard'][0] == 'Yes'):
        data['HasCrCard'][0]= 1
    else:
        data['HasCrCard'][0]= 0
    
    if(data['IsActiveMember'][0] == 'Yes'):
        data['IsActiveMember'][0]= 1
    else:
        data['IsActiveMember'][0]= 0
        
    geo = []
    if(data['Geography'][0] == "France"):
        geo = [0.0,1.0,0.0]
    elif(data['Geography'][0] == "Spain"):
        geo = [0.0,0.0,1.0]
    elif(data['Geography'][0] == "Germany"):
        geo = [1.0,0.0,0.0]
    
    gender = []
    if(data['Gender'][0] == 'Female'):
        gender = [0.0,1.0]
    elif(data['Gender'][0] == 'Male'):
        gender = [1.0,0.0]
        
    
    onhot_data = geo + gender
    onhot_data = pd.DataFrame(onhot_data).T
    onhot_data.columns = ['Geography_Germany','Geography_France','Geography_Spain','Gender_Male','Gender_Female']
    data = data.join(onhot_data)
    
    cat_df = data[["Geography_Germany", "Geography_France","Geography_Spain", "Gender_Male","Gender_Female" ,"HasCrCard","IsActiveMember"]]
    
    X= data.drop(labels=["Geography_France","Geography_Germany","Geography_Spain", "Gender_Female","Gender_Male" ,"HasCrCard","IsActiveMember","Geography","Gender"],axis=1)
    cols = X.columns
    index = X.index
    X = transformers.transform(X)
    X = pd.DataFrame(X, columns = cols, index = index)
    X = pd.concat([X,cat_df], axis = 1)
    
    prediction_val = []
    for i in range(len(X.columns)):
        prediction_val.append(X.iloc[0][i])
    prediction_val = [prediction_val]
    return prediction_val

In [26]:
y=prep_pred(data)
#prediction
pred=model.predict(y)
prediction=pred[0]
#print("Predicted Class:", prediction)

print("Will customer leave the bank?")
if prediction == 1:
  print('Yes, the customer will leave the bank soon.')
elif prediction == 0:
  print('No, the customer is happy with bank services.')
print("\n") 
print("---------------------------------------------------------")

Will customer leave the bank?
No, the customer is happy with bank services.


---------------------------------------------------------


/home/hp/tensorflow_files/virtualenv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/home/hp/tensorflow_files/virtualenv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/home/hp/tensorflow_files/virtualenv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/home/hp/tensorflow_files/virtualenv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
/home/hp/tensorflow_files/virtualenv/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  wa